In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from scipy.stats import kstest, mannwhitneyu
from scipy.stats import ks_2samp
import itertools
# import shap

from tqdm import tqdm

import matplotlib.pyplot as plt
import pingouin as pg
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:

met_file_path = "C:/Users/shrra/Downloads/merged_n586_Alasdair_May21_concatenated_renamed.csv"
met1_df = pd.read_csv(met_file_path)
# print(met1_df.columns)


In [24]:
met1_df_T  = met1_df.T

In [25]:


met1_df_T.reset_index(inplace=True)

# Set the first row as the header
met1_df_T.columns = met1_df_T.iloc[0]
met1_df_T = met1_df_T[1:]

# Reset the index again (if you want to have a clean numeric index)
met1_df_T.reset_index(drop=True, inplace=True)

# Rename the newly created index column (if needed)
met1_df_T.rename(columns={met1_df_T.columns[0]: 'ID'}, inplace=True)


# print(met1_df_T.columns)


In [26]:
met1_df_T['ID'] = met1_df_T['ID'].str.replace('-S.*', '', regex=True)
# print(len(met1_df_T))
# print(met1_df_T.columns)


In [27]:
mas_file_path = "C:/Users/shrra/Downloads/MASH_biomarkers_clinicaldata_notimputed_May30_HH.csv"
mash_df = pd.read_csv(mas_file_path)
mash_df['MASH'] = mash_df['MASH'].map({'MASH': 1, 'non-MASH': 0})
# print(mash_df.columns)


In [28]:
id_df = mash_df[['ID', 'MASH']]


In [29]:
id_df['ID'] = id_df['ID'].str.replace('5-00', '')
id_df['ID'] = id_df['ID'].str.replace('5-0', '')

C:\Users\shrra\AppData\Local\Temp\ipykernel_25244\3199667862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_df['ID'] = id_df['ID'].str.replace('5-00', '')
C:\Users\shrra\AppData\Local\Temp\ipykernel_25244\3199667862.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_df['ID'] = id_df['ID'].str.replace('5-0', '')


In [30]:
df1 = id_df[['ID']]
df2 = met1_df_T[['ID']]


set1 = set(df1['ID'])
set2 = set(df2['ID'])

# Find values in df1['ID'] that are not in df2['ID']
missing_in_df2 = set1 - set2

# Find values in df2['ID'] that are not in df1['ID']
missing_in_df1 = set2 - set1

# # Display the missing values
# print("\nID Values in mash file that are not in metab file")
# print(missing_in_df2)
# print(len(missing_in_df2))
# print("\nID Values in metab file that are not in mash file")
# print(missing_in_df1)
# print(len(missing_in_df1))

In [31]:
Updated_meta_df = pd.merge(id_df, met1_df_T, on=['ID'], how='inner')

In [32]:
KSTest_df = Updated_meta_df


excluded_cols = ["ID"]
KSTest_df= KSTest_df.drop(columns=excluded_cols)

In [33]:


# Assuming KSTest_df is your DataFrame
# Calculate correlation matrix
corr_matrix = KSTest_df.drop('MASH', axis=1).corr().abs()

# Set threshold for high correlation
corr_threshold_low = 0.8
corr_threshold_high = 1.0

# Dictionary to store highly correlated features for each feature
highly_correlated_features = {}

# Loop through the features and find highly correlated features
for feature in corr_matrix.columns:
    highly_corr = corr_matrix[(corr_matrix[feature] > corr_threshold_low) & (corr_matrix[feature] < corr_threshold_high)][feature].index.tolist()
    if highly_corr:
        highly_correlated_features[feature] = highly_corr

# # Print the highly correlated features for each feature
# for feature, correlated in highly_correlated_features.items():
#     print(f"Highly correlated features for {feature}: {correlated}")

# Function to calculate p-value using KS test
def ks_test(feature, target):
    return ks_2samp(feature[target == 0], feature[target == 1]).pvalue

# Perform KS test for each group and keep the feature with the best p-value
selected_features = set()
ks_test_results = {}

# DataFrame to store selected features and their p-values
selected_features_df = pd.DataFrame(columns=['Group', 'Feature', 'p-value'])

# Set to keep track of already selected features
excluded_features = set()

for feature, correlated in highly_correlated_features.items():
    # Filter out already selected features
    candidates = [col for col in correlated + [feature] if col not in excluded_features]
    if not candidates:
        continue
    p_values = {col: ks_test(KSTest_df[col], KSTest_df['MASH']) for col in candidates}
    ks_test_results[feature] = p_values
    best_feature = min(p_values, key=p_values.get)
    selected_features.add(best_feature)
    excluded_features.add(best_feature)
    
    # Add the selected feature and its p-value to the DataFrame
    new_row = pd.DataFrame({'Group': [feature], 'Selected_Feature': [best_feature], 'p-value': [p_values[best_feature]]})
    selected_features_df = pd.concat([selected_features_df, new_row], ignore_index=True)

# # Print KS test results and selected feature from each group
# print("\nKS Test Results and Selected Features from Each Group:\n")
for feature, p_values in ks_test_results.items():
    best_feature = min(p_values, key=p_values.get)
    # print(f"KS test p-values for group starting with {feature}: {p_values}, Best Feature: {best_feature}")

# Final set of features
final_selected_features = list(selected_features)

# # Print the selected features
# print("\nSelected Features:\n", final_selected_features)

# # Display the DataFrame of selected features and their p-values
# print("\nThis df has Selected Features and Their p-values:\n", selected_features_df)


C:\Users\shrra\AppData\Local\Temp\ipykernel_25244\216185983.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_features_df = pd.concat([selected_features_df, new_row], ignore_index=True)


In [34]:
print(len(final_selected_features))
check = set(final_selected_features)
print("So unique would be")
print(len(check))

684
So unique would be
684


In [35]:
count=0
for index, row in selected_features_df.iterrows():
    if row['p-value'] < 0.05:
        count=count+1

print("number of features under 0.05= ",count)

number of features under 0.05=  209


In [36]:
# selected_features_df.rename(columns={'Feature': 'Selected_Feature'}, inplace=True)

In [37]:
# selected_features_df.to_csv('C:/Users/shrra/Downloads/MASH_ML/Metab_group_corr.csv', index=False)